In [1]:
import numpy as np
import pandas as pd
import json
import collections
from collections import defaultdict 
from functools import partial
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import jax

# Global flag to set a specific platform, must be used at startup.
jax.config.update('jax_platform_name', 'gpu')
jax.config.update("jax_debug_nans", True)
jax.config.update("jax_debug_infs", True)

jax.config.update('jax_log_compiles', False)
jax.config.update('jax_check_tracer_leaks', False)

In [2]:
# Good read: https://iq-inc.com/importerror-attempted-relative-import/

import sys
import importlib
from mimicnet import concept
from mimicnet import jax_interface
from mimicnet import dag
from mimicnet import glove
from mimicnet import gram
from mimicnet import ode

importlib.reload(sys.modules['mimicnet.concept'])
importlib.reload(sys.modules['mimicnet.dag'])
importlib.reload(sys.modules['mimicnet.jax_interface'])
importlib.reload(sys.modules['mimicnet.glove'])
importlib.reload(sys.modules['mimicnet.gram'])
importlib.reload(sys.modules['mimicnet.ode'])

<module 'mimicnet.ode' from '/home/asem/GP/MIMIC-SNONET/mimicnet/ode.py'>

In [3]:

KG = dag.CCSDAG()

In [4]:
# multi_visit_mimic_dir = '/home/am8520/GP/ehr-data/mimic3-multi-visit'
multi_visit_mimic_dir = '/home/asem/GP/ehr-data/mimic3-multi-visit'
transformed_mimic_dir = '/home/asem/GP/ehr-data/mimic3-transforms'
mimic_dir = '/home/asem/GP/ehr-data/mimic3-v1.4/physionet.org/files/mimiciii/1.4'
experiments_dir = '/home/asem/GP/ehr-data/mimic3-snonet-exp'
experiment_prefix = 'NOV29'

In [5]:
D_LABITEMS = pd.read_csv(f'{mimic_dir}/D_LABITEMS.csv.gz')
D_ITEMS = pd.read_csv(f'{mimic_dir}/D_ITEMS.csv.gz')
D_TEST = pd.concat([D_LABITEMS, D_ITEMS], join='inner')
test_label_dict = dict(zip(D_TEST.ITEMID, D_TEST.LABEL))
test_cat_dict = dict(zip(D_TEST.ITEMID, D_TEST.CATEGORY))

In [6]:
static_df = pd.read_csv(f'{transformed_mimic_dir}/static_df.csv.gz')
adm_df = pd.read_csv(f'{transformed_mimic_dir}/adm_df.csv.gz')
diag_df = pd.read_csv(f'{transformed_mimic_dir}/diag_df.csv.gz', dtype={'ICD9_CODE': str})
proc_df = pd.read_csv(f'{transformed_mimic_dir}/proc_df.csv.gz', dtype={'ICD9_CODE': str})
test_df = pd.read_csv(f'{transformed_mimic_dir}/test_df.csv.gz')


In [7]:
# Cast columns of dates to datetime64

static_df['DOB'] = pd.to_datetime(static_df.DOB, infer_datetime_format=True).dt.normalize()
adm_df['ADMITTIME'] = pd.to_datetime(adm_df.ADMITTIME, infer_datetime_format=True).dt.normalize()
adm_df['DISCHTIME'] = pd.to_datetime(adm_df.DISCHTIME, infer_datetime_format=True).dt.normalize()
test_df['DATE'] = pd.to_datetime(test_df.DATE, infer_datetime_format=True).dt.normalize()

In [8]:
static_df.ETHNIC_GROUP.nunique()

7

In [9]:
test_df

,SUBJECT_ID,ITEMID,DATE,VALUE
0,17,50852,2134-12-22,-1.201339
1,17,50861,2134-12-22,-0.809370
2,17,50862,2134-12-22,1.634410
3,17,50863,2134-12-22,-1.452596
4,17,50867,2134-12-22,-0.608894
...,...,...,...,...
4546204,99982,227456,2157-02-22,1.794272
4546205,99982,227457,2157-02-22,-0.723781
4546206,99982,227465,2157-02-22,2.247434
4546207,99982,227466,2157-02-22,-0.029395


In [10]:
patients = concept.Subject.to_list(static_df, adm_df, diag_df, proc_df, test_df)

In [11]:
len(patients)

4434

In [12]:
# ehr_dfs = concept.Subject.to_df(patients)

In [13]:
# def df_eq(df1, df2):
#     df11 = df1.sort_values(by=df1.columns.tolist()).reset_index(drop=True)
#     df22 = df2.sort_values(by=df2.columns.tolist()).reset_index(drop=True)
#     return df11[df22.columns].equals(df22)

# all(map(df_eq, ehr_dfs, [static_df, adm_df, diag_df, proc_df, test_df]))

### [FORK] Skip the cell below to load the jaxified data from a stored file on disc

In [14]:
# subjects_interface = jax_interface.SubjectJAXInterface(patients, set(test_df.ITEMID), KG)
# import pickle
# with open(f'{experiments_dir}/{experiment_prefix}_subjects_interface.pkl', 'wb') as pickleFile:
#     pickle.dump(subjects_interface, pickleFile)

In [15]:
import pickle
with open(f'{experiments_dir}/{experiment_prefix}_subjects_interface.pkl', 'rb') as pickleFile:
    subjects_interface = pickle.load(pickleFile)

## GloVe Initialization

In [16]:
%load_ext autoreload
%autoreload 2

In [17]:
import logging
logging.basicConfig(level=logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [18]:
glove_args = {
    'diag_idx': subjects_interface.diag_multi_ccs_idx,
    'proc_idx': subjects_interface.proc_multi_ccs_idx,
    'ccs_dag': KG,
    'subjects': patients,
    'diag_vector_size': 100,
    'proc_vector_size': 60,
    'iterations': 30,
    'window_size_days': 2 * 365
}

diag_glove_rep, proc_glove_rep = glove.glove_representation(**glove_args)

In [19]:
print(f'#point_indices: {len(subjects_interface.nth_points)}')
print(f'#total_points: {sum(len(points) for n, points in subjects_interface.nth_points.items())}')

#[len(points) for n, points in subjects_interface.nth_points.items()]

#point_indices: 1085
#total_points: 129334


## GRAM objects

In [20]:
from datetime import datetime
daily_tracer = "/tmp/tensorboard/"+ datetime.now().strftime("%Y%m%d-%H%M%S") 
print(daily_tracer)

/tmp/tensorboard/20211202-222535


In [21]:


logs = '/tmp/tensorboard/20210708-182059'
#server = jax.profiler.start_server(9999)

In [22]:
       
config = {
    'gram_config': {
        'diag': {
            'ccs_dag': KG,
            'code2index': subjects_interface.diag_multi_ccs_idx,
            'attention_method': 'tanh', #l2, tanh
            'attention_dim': 100,
            'ancestors_mat': subjects_interface.diag_multi_ccs_ancestors_mat,
            'basic_embeddings': diag_glove_rep
        },
        'proc': {
            'ccs_dag': KG,
            'code2index': subjects_interface.proc_multi_ccs_idx,
            'attention_method': 'tanh',
            'attention_dim': 60,
            'ancestors_mat': subjects_interface.proc_multi_ccs_ancestors_mat,
            'basic_embeddings': proc_glove_rep
        }
    },
    'model': {
        'ode_dyn': 'mlp', # gru, mlp
        'state_size': 50,
        'numeric_hidden_size': 40,
        'bias': True
    },
    'training': {
        'train_validation_split': 0.8,
        'batch_size': 12,
        'epochs': 5,
        'lr': 1e-3,
        'diag_loss': 'balanced_focal', # balanced_focal, bce
        'tay_reg': None, # Order of regularized derivative of the dynamics function (None for disable).
        'loss_mixing': {
            'num_alpha': 0.1,
            'diag_alpha': 0.1,
            'ode_alpha': 1e-3,
            'l1_reg': 1e-6,
            'l2_reg': 1e-5,
            'dyn_reg': 1e-5
        },
        'eval_freq': 10,
        'save_freq': 100,
        'save_params_prefix': None
    }
}


In [23]:
diag_gram = gram.DAGGRAM(**config['gram_config']['diag'])

DEBUG:absl:Initializing backend 'interpreter'
DEBUG:absl:Backend 'interpreter' initialized
DEBUG:absl:Initializing backend 'cpu'
DEBUG:absl:Backend 'cpu' initialized
DEBUG:absl:Initializing backend 'tpu_driver'
INFO:absl:Unable to initialize backend 'tpu_driver': Not found: Unable to find driver in registry given worker: 
DEBUG:absl:Initializing backend 'gpu'
DEBUG:absl:Backend 'gpu' initialized
DEBUG:absl:Initializing backend 'tpu'
INFO:absl:Unable to initialize backend 'tpu': Invalid argument: TpuPlatform is not available.


In [24]:
proc_gram = gram.DAGGRAM(**config['gram_config']['proc'])

## GRU-ODE-Bayes

In [ ]:
import random
%load_ext autoreload
%autoreload 2
from absl import logging
logging.set_verbosity(logging.INFO)

#with jax.profiler.trace(logs):
res = ode.train_ehr(subject_interface=subjects_interface,
                diag_gram=diag_gram,
                proc_gram=proc_gram,
                rng=random.Random(42),
                **config['model'],
                **config['training'],
                verbose_debug=False,
                shape_debug=False,
                nan_debug=False,
                memory_profile=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


INFO:ode:#params: 293044
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
info retrieval:   0%|          | 1/1477 [04:19<106:35:09, 259.97s/it]                 INFO:ode:
                        Training    Validation
prejump_num_loss       14.317818    917.747375
postjump_num_loss     890.755676  61111.664062
prejump_diag_loss       0.002124      0.076024
postjump_diag_loss      0.002105      0.074312
num_loss              101.961604   6937.139044
diag_loss               0.002122      0.075853
ode_loss                0.104082      7.012916
l1_loss             14247.754883  14247.754883
l1_loss_per_point      31.732194      0.562463
l2_loss              1785.656128   1785.656128
l2_loss_per_point       3.976962      0.070493
dyn_loss                 14200.0      775924.0
dyn_loss_per_week      10.344469     7.9545236
loss                 0.104449585      7.013223
INFO:ode:
                          Training     Valdation
accuracy                  0.506583      0.486989
recall        

info retrieval:   3%|▎         | 41/1477 [33:01<10:43:15, 26.88s/it]                 INFO:ode:
                        Training    Validation
prejump_num_loss       16.301437    978.553467
postjump_num_loss     781.436096  49125.906250
prejump_diag_loss       0.001322      0.049367
postjump_diag_loss      0.001341      0.049680
num_loss               92.814903   5793.288745
diag_loss               0.001324      0.049398
ode_loss                0.094137      5.842637
l1_loss             14348.020508  14348.020508
l1_loss_per_point      46.584482      0.566421
l2_loss              1760.824951   1760.824951
l2_loss_per_point       5.716964      0.069513
dyn_loss                  9208.0      782686.0
dyn_loss_per_week       8.288029      8.023846
loss                  0.09451132      5.842947
INFO:ode:
                          Training     Valdation
accuracy                  0.804049      0.799993
recall                    0.830409      0.780383
npv                       0.993489      0.9

INFO:ode:
                          Training     Valdation
accuracy                  0.793427      0.790047
recall                    0.768340      0.802502
npv                       0.986937      0.990483
specificity               0.794566      0.789568
precision                 0.145150      0.127777
f1-score                  0.244172      0.220453
tp                        0.033367      0.029687
tn                        0.760060      0.760360
fp                        0.196512      0.202647
fn                        0.010060      0.007306
points_count            229.000000  25331.000000
odeint_weeks_per_visit    4.089832      3.850815
nfe_per_point             0.000000      0.000000
nfe_per_week              0.000000      0.000000
nfex1000                  0.000000      0.000000
INFO:ode:
    Training(pre)  Training(post)  Validation(pre)  Validation(post)
p0       0.000000        0.000000         0.009000          0.006429
p1       0.000000        0.000000         0.013609        

INFO:ode:
    Training(pre)  Training(post)  Validation(pre)  Validation(post)
p0       0.000000        0.000000         0.007715          0.010929
p1       0.035714        0.107143         0.024556          0.044970
p2       0.191176        0.132353         0.364190          0.288063
p3       0.901961        0.882353         0.714646          0.747790
p4       1.000000        0.963636         0.998612          0.991396
info retrieval:   9%|▉         | 131/1477 [1:34:27<10:44:32, 28.73s/it]                 INFO:ode:
                        Training    Validation
prejump_num_loss       16.610407    995.001038
postjump_num_loss     618.306885  37928.191406
prejump_diag_loss       0.000999      0.046697
postjump_diag_loss      0.001009      0.047370
num_loss               76.780055   4688.320074
diag_loss               0.001000      0.046764
ode_loss                0.077779      4.735038
l1_loss             14359.715820  14359.715820
l1_loss_per_point      39.558446      0.566883
l2_loss 

...

info retrieval:  56%|█████▌    | 821/1477 [9:27:17<4:11:30, 23.00s/it]                 INFO:ode:
                        Training    Validation
prejump_num_loss       17.329493   1094.135010
postjump_num_loss     585.177917  36549.941406
prejump_diag_loss       0.001440      0.050245
postjump_diag_loss      0.001466      0.049459
num_loss               74.114335   4639.715649
diag_loss               0.001443      0.050166
ode_loss                0.075556      4.689832
l1_loss             19584.375000  19584.375000
l1_loss_per_point      71.475821      0.773139
l2_loss              2458.484863   2458.484863
l2_loss_per_point       8.972572      0.097054
dyn_loss                  9386.0      892168.0
dyn_loss_per_week       8.434146       9.14622
loss                 0.076014526     4.6901855
INFO:ode:
                          Training     Valdation
accuracy                  0.766308      0.765392
recall                    0.830097      0.790294
npv                       0.991248      0

INFO:ode:
                          Training     Valdation
accuracy                  0.767438      0.760373
recall                    0.812500      0.798574
npv                       0.990534      0.989907
specificity               0.765679      0.758906
precision                 0.119188      0.112876
f1-score                  0.207881      0.197795
tp                        0.030516      0.029542
tn                        0.736922      0.730832
fp                        0.225520      0.232175
fn                        0.007042      0.007451
points_count            240.000000  25331.000000
odeint_weeks_per_visit    4.856548      3.850815
nfe_per_point             0.000000      0.000000
nfe_per_week              0.000000      0.000000
nfex1000                  0.000000      0.000000
INFO:ode:
    Training(pre)  Training(post)  Validation(pre)  Validation(post)
p0       0.020408        0.020408         0.018965          0.012536
p1       0.071429        0.119048         0.082249        

INFO:ode:
    Training(pre)  Training(post)  Validation(pre)  Validation(post)
p0       0.037037        0.037037         0.012858          0.014143
p1       0.178571        0.214286         0.118935          0.139053
p2       0.275862        0.344828         0.327345          0.299026
p3       0.320000        0.320000         0.458333          0.428977
p4       0.967742        1.000000         0.991119          0.985012
info retrieval:  62%|██████▏   | 911/1477 [10:30:58<3:37:56, 23.10s/it]                 INFO:ode:
                        Training    Validation
prejump_num_loss       13.108520   1059.991943
postjump_num_loss     436.391235  35106.246094
prejump_diag_loss       0.001430      0.049438
postjump_diag_loss      0.001460      0.048819
num_loss               55.436791   4464.617358
diag_loss               0.001433      0.049376
ode_loss                0.056868      4.513944
l1_loss             19842.220703  19842.220703
l1_loss_per_point      42.763407      0.783318
l2_loss 

info retrieval:  64%|██████▍   | 951/1477 [11:00:05<4:38:12, 31.74s/it]                 INFO:ode:
                        Training    Validation
prejump_num_loss       17.980234   1056.454224
postjump_num_loss     612.331848  34982.953125
prejump_diag_loss       0.001095      0.049765
postjump_diag_loss      0.001101      0.048395
num_loss               77.415396   4449.104114
diag_loss               0.001096      0.049628
ode_loss                0.078510      4.498683
l1_loss             19902.640625  19902.640625
l1_loss_per_point      56.864688      0.785703
l2_loss              2520.234619   2520.234619
l2_loss_per_point       7.200670      0.099492
dyn_loss                 11344.0      895228.0
dyn_loss_per_week      10.278023      9.177589
loss                 0.078926794     4.4990373
INFO:ode:
                          Training     Valdation
accuracy                  0.759977      0.763237
recall                    0.789744      0.805221
npv                       0.989130      

INFO:ode:
                          Training     Valdation
accuracy                  0.759585      0.758147
recall                    0.820144      0.809573
npv                       0.993411      0.990419
specificity               0.757893      0.756171
precision                 0.086495      0.113117
f1-score                  0.156486      0.198499
tp                        0.022300      0.029949
tn                        0.737285      0.728198
fp                        0.235524      0.234809
fn                        0.004890      0.007044
points_count            305.000000  25331.000000
odeint_weeks_per_visit    4.580328      3.850815
nfe_per_point             0.000000      0.000000
nfe_per_week              0.000000      0.000000
nfex1000                  0.000000      0.000000
INFO:ode:
    Training(pre)  Training(post)  Validation(pre)  Validation(post)
p0       0.064516        0.032258         0.028608          0.024751
p1       0.269231        0.230769         0.194083        

#### Possible modifications:
- Add more layers to the adjustment function
- Use days instead of weeks for odeint